In [2]:
import pinecone as pn
pn.__version__

'2.0.13'

## Index Creation

> An index is the highest-level organizational unit of vector data in Pinecone. It accepts and stores vectors, serves queries over the vectors it contains, and does other vector operations over its contents. Each index runs on at least one pod.

In [13]:
def initialize_pinecone():
    """
    To use Pinecone, one must have an API key.
    By default, ENVIRONMENT is us-west1-gcp.
    :return:None
    """
    PINECONE_API_KEY = "XXXXXXXXXXXXXXXXXXXXXXXXXXX"
    pinecone.init(api_key=PINECONE_API_KEY, environment="us-west1-gcp")


def create_pinecone_index(pinecone_index_name):
    """
     Create an index named $pinecone_index_name that will perform approximate nearest-neighbor search using cosine distance for 768-dimensional vectors
    :param pinecone_index_name
    :return:None
    """
    if pinecone_index_name in pinecone.list_indexes():
        # pinecone.delete_index(pinecone_index_name)
        return {"ERROR": "Index Already exist", "ErrorCode": 400}, 400
    pinecone.create_index(
        name=pinecone_index_name,
        index_type="approximated",
        dimension=512,
        metric="cosine",
    )

## Connect to Index

In [6]:
def connect_pinecone_index(pinecone_index_name):
    """
    Connection to the pinecone index
    :param pinecone_index_name
    :return: pinecone index
    """
    initialize_pinecone()
    pinecone_index = pinecone.Index(index_name=pinecone_index_name)
    return pinecone_index

In [14]:
initialize_pinecone()
pinecone_index_name = "myfirstindex"
create_pinecone_index(pinecone_index_name)
index = connect_pinecone_index(pinecone_index_name)

## Configurations

### 1. List all your Pinecone indexes:

In [15]:
pinecone.list_indexes()

['myfirstindex']

### 2 . Get the configuration and current status of an index named "pinecone-index":

In [16]:
pinecone.describe_index(pinecone_index_name)

IndexDescription(name='myfirstindex', metric='cosine', replicas=1, dimension=512.0, shards=1, pods=1, pod_type='p1', status={'ready': True, 'state': 'Ready'}, metadata_config=None, source_collection='')

### Performance-Tuning
As a starting point, you could choose
>* shards=2, 
>* pods=2, 
>* podtype='p1.x1'


Then, add replicas to achieve QPS. More shards will also allow you to increase your batch upsert count >100 depending on other factors mentioned here: https://www.pinecone.io/docs/performance-tuning/#performance-tuning

### 3.  Check index size for each namespace


In [17]:
index.describe_index_stats()

{'dimension': 512,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

### P1 pods
* These performance-optimized pods provide very low query latencies, but hold fewer vectors per pod than s1 pods. They are ideal for applications with low latency requirements (<100ms).

* Each p1 pod has enough capacity for around `1M vectors of 768 dimensions.`
* However, unlike with p1 and s1 pods, filtering searches of indexes on p2 pods does not improve query latency.

### 4. Increasing the pod size for an index

In [ ]:
pinecone.configure_index(pinecone_index_name, pod_type="p1.x1")

## Limits

1. Max vector dimensionality is 20,000.

2. Max size for an upsert request is 2MB.

3. Max value for top_k, the number of results to return, is 10,000.

4. Max value for top_k for queries with include_metadata=True or include_data=True is 1,000

5. Max vectors per fetch or delete request is 1,000.

6. Each p1 pod has enough capacity for 1M vectors with 768 dimensions.

7. Each s1 pod has enough capacity for 5M vectors with 768 dimensions.

8. Max metadata size per vector is 10 KB.